# Interactively segmenting fibers
InSegt-py is a py version of [InSegt](https://github.com/vedranaa/InSegt). Basic variant of InSegt is described in our paper [Content-based Propagation of User Markings for Interactive Segmentation of Patterned Images](http://openaccess.thecvf.com/content_CVPRW_2020/papers/w57/Dahl_Content-Based_Propagation_of_User_Markings_for_Interactive_Segmentation_of_Patterned_CVPRW_2020_paper.pdf), CVPRW 2020. But InSegt has evolved, so please check the demos and notebooks for the updated version.

This is an example of interactive image segmentation with InSegt. Here we usa a model which builds dictionay by clustering image features in a k-means tree. For features we use Gaussian derivatives, where we use a few different values for the standard deviation of the Gaussian kernel. Furthermore, the model is made multi-scale by building and including sub-models operating on a downscaled version of the input image. 

In this example, we use a sub-volume of a fiber composite material. The study of the data is described in the article [Individual fibre inclination segmentation from X-ray computed tomography using principal component analysis](https://journals.sagepub.com/doi/epub/10.1177/00219983211052741), Journal of Composite Materials 2021. The data is available from [Zenodo](https://zenodo.org/records/5483719).

## Import packages
Most importantly, you need `insegt` and `insegtpy.models`. You also need to be able to read in the image (for example using `PIL`), and show the result (for example using `matplotlib`).

In [ ]:
# Import the modules needed 
import insegtpy
import insegtpy.models
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import tifffile
import urllib.request 
import skimage.filters
import skimage.feature
import PIL.Image

%matplotlib widget

In [ ]:
#%% Load image from tiff repository - choose between 'Mock' and 'UD'

name = 'UD' # 'Mock' or 'UD'
vol_name = {'Mock': 'Mock_cropped.tif', 'UD': 'UD_cropped.tif'}

url_in = 'https://data.qim.dk/InSegt_data/3D/' + vol_name[name] 
urllib.request.urlretrieve(url_in, 'Volume.tif')
V = tifffile.imread('Volume.tif')

fig, ax = plt.subplots(1, 3, figsize=(10, 5))
ax[0].imshow(V[100], cmap='gray')
ax[0].set_title('XY plane')
ax[1].imshow(V[:,100], cmap='gray')
ax[1].set_title('XZ plane')
ax[2].imshow(V[:,:,100], cmap='gray')
ax[2].set_title('YZ plane')
plt.show()

dir_out = 'model/' # Output directory - potentially change this to your own directory
if not os.path.exists(dir_out):
    os.makedirs(dir_out)

In [ ]:
#%% Build the model - only possible to build a small model from such a small image
image_train = V[100]
model = insegtpy.models.gauss_features_segmentor(image_train, 
                                   branching_factor = 5, 
                                   number_layers = 5,
                                   number_training_vectors = 20000,
                                   features_sigma = [1, 2],
                                   propagation_size = 9, 
                                   scales=[1, 0.75],
                                   propagation_repetitions=2)

In [ ]:
# Start the annotation GUI
ex = insegtpy.insegt(image_train, model, saveAddress=dir_out, savePrefix=name)

In [ ]:
# Show probabilities of segmentation

ncls = ex.probabilities.shape[0]
fig, ax = plt.subplots(1,ncls)
for i in range(ncls):
    ax[i].imshow(ex.probabilities[i])


In [ ]:
# Compare segmentation of training and test image

prob = ex.probabilities
seg = insegtpy.utils.segment_probabilities(prob)

# Another slide
image_test = V[0]

prob_new = model.segment_new(image_test)
seg_new = insegtpy.utils.segment_probabilities(prob_new)
seg_new[seg_new==0] = 1 # if some pixels are set to zero

fig, ax = plt.subplots(2, 2, sharex = True, sharey = True )
ax[0][0].imshow(image_train)
ax[1][0].imshow(seg)
ax[1][0].set_title('Train')
ax[0][1].imshow(image_test)
ax[1][1].imshow(seg_new)
ax[1][1].set_title('Test')
plt.show()


In [ ]:
# Save model to file
model_file_name = 'segmentation_model_ud.pkl'
with open(os.path.join(dir_out, model_file_name), 'wb') as f:
    pickle.dump(model, f)


In [ ]:
# Segment all files in a repository
V_prob = []
V_seg = []

for im in V:
    prob = model.segment_new(im)
    seg = insegtpy.utils.segment_probabilities(prob)
    seg[seg==0] = 1
    V_prob.append(prob[2])
    V_seg.append(seg)
V_prob = np.array(V_prob)
V_seg = np.array(V_seg)


In [ ]:
# Plot center points of the segmented fibers

pts = []
for i, porb in enumerate(V_prob):
    prob = skimage.filters.gaussian(porb, 2)
    coords = skimage.feature.peak_local_max(prob, min_distance=5, threshold_abs=0.1)
    pts.append(np.append(coords-1, i*np.ones((coords.shape[0],1)), axis=1))
pts = np.vstack(pts)
fig = plt.figure(figsize=(7,7))
ax = plt.axes(projection='3d')
ax.plot3D(pts[:,0], pts[:,1], pts[:,2], 'b.', alpha=0.1)


## Load saved model 
Assumes that the volume has been loaded

In [ ]:
# Load model from file
model_file_name = 'segmentation_model_ud.pkl'
with open(os.path.join(dir_out, model_file_name), 'rb') as f:
    model = pickle.load(f)

V_prob = []
V_seg = []

for im in V:
    prob = model.segment_new(im)
    seg = insegtpy.utils.segment_probabilities(prob)
    seg[seg==0] = 1
    V_prob.append(prob[2])
    V_seg.append(seg)
V_prob = np.array(V_prob)
V_seg = np.array(V_seg)

# Plot center points of the segmented fibers

pts = []
for i, porb in enumerate(V_prob):
    prob = skimage.filters.gaussian(porb, 2)
    coords = skimage.feature.peak_local_max(prob, min_distance=5, threshold_abs=0.1)
    pts.append(np.append(coords-1, i*np.ones((coords.shape[0],1)), axis=1))
pts = np.vstack(pts)
fig = plt.figure(figsize=(7,7))
ax = plt.axes(projection='3d')
ax.plot3D(pts[:,0], pts[:,1], pts[:,2], 'b.', alpha=0.1)


In [ ]:
# Start the annotation GUI with pre-annotated labels
image_train = V[100]
labels = np.array(PIL.Image.open(dir_out + name + '_annotations_index.png'))
ex = insegtpy.insegt(image_train, model, saveAddress=dir_out, labels=labels, savePrefix=name)